### Load Classifier

In [6]:
import torchvision 
import torch 
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    return model.eval()


filepath = "torch_classifier.pth" #Pre-trained resnet50 ile fine-tunning yapılmış model, accuracy: %99 on the test set.
model = load_checkpoint(filepath)

train_transforms = transforms.Compose([
    #transforms.Grayscale(num_output_channels=3), renkler önemli gray scale yapma
    transforms.Resize((224,224)), # resnet girişi
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ])


### Real Time Detecting

In [8]:
import numpy as np
import cv2
import cvlib as cv
import sys
from PIL import Image
import numpy as np

extra_pixel = 150

#Parameters for Texts
font_scala=0.6
renk=(0,0,255)
font_tip=cv2.FONT_HERSHEY_SIMPLEX

#Save Video
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('real-time-detecting.avi',fourcc, 20.0, (640,480))

#Activate Capturing
cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 800)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1000)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    #out.write(frame)
    
    
    
    
    
    #Detect Faces
    faces, confidences = cv.detect_face(frame)
    
    
    if(len(faces)!=0): #If the face is detected.
        
        #cv2.putText(frame,"Face is detected.",(25,40),font_tip,font_scala,renk)
    
        for face in faces:
            (startX,startY) = face[0],face[1]
            (endX,endY) = face[2],face[3]
            current_face = frame[startX-extra_pixel:endX+extra_pixel,startY-extra_pixel:endY+extra_pixel]
            
            pil_image = Image.fromarray(current_face, mode = "RGB")#numpy to pil image because of torch model
            pil_image = train_transforms(pil_image)
            image = pil_image.unsqueeze(0)
            
    
            result = model(image).data.numpy()
            
            #print(result)
            
            
            if result[0][0] > result[0][1]:#masked
                cv2.putText(frame,"Masked",(350,40),font_tip,font_scala,renk)
                cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)
            else:#unmasked
                cv2.putText(frame,"Unmasked",(350,40),font_tip,font_scala,renk)
                cv2.rectangle(frame, (startX,startY), (endX,endY), (255,0,0), 2)
            
            out.write(frame)
    
    else: # If there is no face.    
        cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)
        #cv2.putText(frame,"There is no face.",(25,40),font_tip,font_scala,renk)
    
    
    cv2.imshow('Medical Mask Detector', frame)
    
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
out.release()
cv2.destroyAllWindows()

### Detecting on the videos saved before

In [5]:
import numpy as np
import cv2
import cvlib as cv
import sys
from PIL import Image
import numpy as np


#Save Video
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('saved-video-before.avi',fourcc, 20.0, (450,1800))



#Parameters for Texts
font_scala=0.6
renk=(0,0,255)
font_tip=cv2.FONT_HERSHEY_SIMPLEX

cap = cv2.VideoCapture('output.avi')

while(cap.isOpened()):
    ret, frame = cap.read()
    
    pil_image = Image.fromarray(frame, mode = "RGB")#numpy to pil image because of torch model
    pil_image = train_transforms(pil_image)
    image = pil_image.unsqueeze(0)
    
    
    #Detect Faces
    faces, confidences = cv.detect_face(frame)
    
    if(len(faces)!=0): #If the face is detected.
        
        #cv2.putText(frame,"Face is detected.",(25,40),font_tip,font_scala,renk)
    
        for face in faces:
            (startX,startY) = face[0],face[1]
            (endX,endY) = face[2],face[3]
            current_face = frame[startX:endX,startY:endY]
            
    
            result = model(image).data.numpy()
            
            #print(result)
            
            
            if result[0][0] > result[0][1]:#masked
                cv2.putText(frame,"Masked",(350,40),font_tip,font_scala,renk)
                cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)
            else:#unmasked
                cv2.putText(frame,"Unmasked",(350,40),font_tip,font_scala,renk)
                cv2.rectangle(frame, (startX,startY), (endX,endY), (255,0,0), 2)
            
            out.write(frame)
    
    else: # If there is no face.    
        cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)
        #cv2.putText(frame,"There is no face.",(25,40),font_tip,font_scala,renk)
    

    cv2.imshow('Medical Mask Detector',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()